Project: E-Commerce Data Lakehouse
Stage 3: Data Governance, Access History & Audit Trail

This notebook focuses on implementing Delta Lake Governance features, including metadata management, version auditing, and time-travel traceability for the Silver layer.

In [0]:
%sql
USE rishikesh_db;

In [0]:
%sql
DESCRIBE HISTORY silver_sales_enriched;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2026-02-06T12:06:50.000Z,72178728963472,rishikeshmate09@gmail.com,SET TBLPROPERTIES,"Map(properties -> {""project"":""E-Commerce Lakehouse"",""layer"":""Silver"",""description"":""Cleaned and deduplicated sales data"",""update_frequency"":""Daily""})",null,List(4210502363580170),0206-115824-urmivehi-v2n,4,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
4,2026-02-06T12:05:53.000Z,72178728963472,rishikeshmate09@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(4210502363580170),0206-115824-urmivehi-v2n,3,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 32852, numDeletionVectorsRemoved -> 0, numOutputRows -> 4000, numOutputBytes -> 33171)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
3,2026-01-30T10:46:50.000Z,72178728963472,rishikeshmate09@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(3107169240403252),0130-104030-xnmkmskq-v2n,2,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 32852, numDeletionVectorsRemoved -> 0, numOutputRows -> 4000, numOutputBytes -> 32852)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
2,2026-01-30T10:44:53.000Z,72178728963472,rishikeshmate09@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(3107169240403252),0130-104030-xnmkmskq-v2n,1,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 32846, numDeletionVectorsRemoved -> 0, numOutputRows -> 4000, numOutputBytes -> 32852)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
1,2026-01-30T10:43:39.000Z,72178728963472,rishikeshmate09@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(3107169240403252),0130-104030-xnmkmskq-v2n,0,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 32846, numDeletionVectorsRemoved -> 0, numOutputRows -> 4000, numOutputBytes -> 32846)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
0,2026-01-30T10:42:24.000Z,72178728963472,rishikeshmate09@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(3107169240403252),0130-104030-xnmkmskq-v2n,null,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 4000, numOutputBytes -> 32846)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


In [0]:
from pyspark.sql.functions import lit

# 1. Load the current data
df = spark.table("silver_sales_enriched")

# 2. Add the new governance/audit column
df_updated = df.withColumn("governance_check", lit("PASSED_FEB_06"))

# 3. Write with 'mergeSchema' enabled to allow the new column
df_updated.write.format("delta") \
  .mode("overwrite") \
  .option("mergeSchema", "true") \
  .saveAsTable("silver_sales_enriched")

In [0]:
# Query Version 0 (before we added the governance_check column)
df_old_version = spark.read.format("delta").option("versionAsOf", 0).table("silver_sales_enriched")

display(df_old_version)

order_id,customer_id,customer_name,sale_amount,silver_load_time
317568014,null,"otbda , outside the box digital agency ,",100.00,2026-01-30T10:42:18.791Z
317568015,null,denevi digital imaging,100.00,2026-01-30T10:42:18.791Z
317568016,null,rpm optoelectronics,100.00,2026-01-30T10:42:18.791Z
317568017,null,"non typical, inc.",100.00,2026-01-30T10:42:18.791Z
317568018,null,als deli,100.00,2026-01-30T10:42:18.791Z
317568019,null,"MORENO, SALVADOR C",100.00,2026-01-30T10:42:18.791Z
317568020,null,"TURNER ALSTON, DENISE",100.00,2026-01-30T10:42:18.791Z
317568021,null,"BOYLAN, HENRI M",100.00,2026-01-30T10:42:18.791Z
317568022,null,guardian sprinklers inc,100.00,2026-01-30T10:42:18.791Z
317568023,null,productive electronics llc,100.00,2026-01-30T10:42:18.791Z


In [0]:
%sql
ALTER TABLE silver_sales_enriched SET TBLPROPERTIES (
  'project' = 'E-Commerce Lakehouse',
  'layer' = 'Silver',
  'description' = 'Cleaned and deduplicated sales data',
  'update_frequency' = 'Daily'
);

-- Verify the properties were added
DESCRIBE EXTENDED silver_sales_enriched;

col_name,data_type,comment
order_id,bigint,null
customer_id,bigint,null
customer_name,string,null
sale_amount,"decimal(10,2)",null
silver_load_time,timestamp,null
governance_check,string,null
,,
# Delta Statistics Columns,,
Column Names,"order_id, governance_check, sale_amount, silver_load_time, customer_id, customer_name",
Column Selection Method,first-32,
